### Useful import

In [17]:
import pandas as pd
import numpy as np
import glob
import re
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import tree
import random

In [6]:
path_shots_LNA = "Data/Team/LNA_Regular_Season/shots/*"
path_shots_LNB = "Data/Team/LNB_Regular_Season/shots/*"
df_shots = []


for file in glob.iglob(path_shots_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_shots.append(frame)

for file in glob.iglob(path_shots_LNB):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_shots.append(frame)
    
df_shots = pd.concat(df_shots)

df_shots.drop(df_shots.columns[[0]], axis=1, inplace=True)
df_shots.reset_index(inplace=True)
df_shots = df_shots.drop(df_shots[df_shots.Team == 'Durchschnitt'].index)
df_shots.set_index(['Team','Year'], inplace=True)
df_shots.sort_index(inplace=True)
df_shots.head(5)

index  GP   GF SOG S% SOG/GP
Team             Year                              
EHC Basel Sharks 0910      0  45  111   0  -      0
                 1011      0  45  138   0  -      0
                 1112      0  45  132   0  -      0
                 1213      0  50  138   0  -      0
                 1314      0  45  144   0  -      0

In [7]:
path_powerkill_LNA = "Data/Team/LNA_Regular_Season/powerkill/*"
path_powerkill_LNB = "Data/Team/LNB_Regular_Season/powerkill/*"

df_powerkill = []

for file in glob.iglob(path_powerkill_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerkill.append(frame)

for file in glob.iglob(path_powerkill_LNB):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerkill.append(frame)
    
df_powerkill = pd.concat(df_powerkill)

df_powerkill.drop(df_powerkill.columns[[0]], axis=1, inplace=True)
df_powerkill.reset_index(inplace=True)
df_powerkill = df_powerkill.drop(df_powerkill[df_powerkill.Team == 'Durchschnitt'].index)
df_powerkill.set_index(['Team','Year'], inplace=True)
df_powerkill.sort_index(inplace=True)

df_powerkill.head(10)

index  GP  PK SI PK SI/GP  PK GA PK GA/GP    PK%  \
Team             Year                                                     
EHC Basel Sharks 0910      9  45    224     4.98     53     1.18  76.34   
                 1011      2  45    211     4.69     38     0.84  81.99   
                 1112      9  45    212     4.71     45        1  78.77   
                 1213      7  50    232     4.64     46     0.92  80.17   
                 1314      7  45    209     4.64     41     0.91  80.38   
EHC Biel         0809      7  50    306     6.12     55      1.1  82.03   
                 0910      4  50    266     5.32     45      0.9  83.08   
                 1011     11  50    201     4.02     47     0.94  76.62   
                 1112      9  50    200        4     35      0.7   82.5   
                 1213      4  50    198     3.96     33     0.66  83.33   

                          PKT PK time/GP  SHG PK GF/GP  
Team             Year                                   
EHC Basel Sharks 0910  332:41      07:23    2     0.04  
                 1011  330:49      07:21    6     0.13  
                 1112  312:50      06:57    9      0.2  
                 1213  355:41      07:06    8     0.16  
                 1314  324:50      07:13    8     0.18  
EHC Biel         0809  461:34      09:13    3     0.06  
                 0910  405:36      08:06    7     0.14  
                 1011  323:29      06:28    3     0.06  
                 1112  329:47      06:35    1     0.02  
                 1213  317:33      06:21    7     0.14

In [8]:
path_goals_LNA = "Data/Team/LNA_Regular_Season/goals/*"
path_goals_LNB = "Data/Team/LNB_Regular_Season/goals/*"

df_goals = []

for file in glob.iglob(path_goals_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_goals.append(frame)

for file in glob.iglob(path_goals_LNB):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[35:39]
    df_goals.append(frame)
    
df_goals = pd.concat(df_goals)

df_goals.drop(df_goals.columns[[0]], axis=1, inplace=True)
df_goals.reset_index(inplace=True)
df_goals = df_goals.drop(df_goals[df_goals.Team == 'Durchschnitt'].index)
df_goals.set_index(['Team','Year'], inplace=True)
df_goals.sort_index(inplace=True)

df_goals.head(10)

index  GP   GF GF/GP   GA GA/GP  GF even GF even/GP  \
Team             Year                                                        
EHC Basel Sharks 0910     10  45  111  2.47  180  4.00       79       1.76   
                 1011      8  45  138  3.07  136  3.02       98       2.18   
                 1112      6  45  132  2.93  142  3.16       96       2.13   
                 1213      9  50  138  2.76  168  3.36       93       1.86   
                 1314      4  45  144   3.2  162   3.6       93       2.07   
EHC Biel         0809      0  50  126  2.52  213  4.26       94       1.88   
                 0910     10  50  134  2.68  182  3.64       80        1.6   
                 1011      8  50  135   2.7  173  3.46       89       1.78   
                 1112     10  50  114  2.28  128  2.56       80        1.6   
                 1213      5  50  160   3.2  161  3.22      112       2.24   

                       GA even GA even/GP  PPG PPG/GP  PP GA PP GA/GP  SHG  \
Team             Year                                                        
EHC Basel Sharks 0910      117       2.60   33   0.73      8     0.18    2   
                 1011       91       2.02   33   0.73      3     0.07    6   
                 1112       88       1.96   27    0.6      5     0.11    9   
                 1213      117       2.34   35    0.7      4     0.08    8   
                 1314      112       2.49   40   0.89      7     0.16    8   
EHC Biel         0809      145        2.9   29   0.58     13     0.26    3   
                 0910      129       2.58   44   0.88      5      0.1    7   
                 1011      116       2.32   39   0.78      6     0.12    3   
                 1112       88       1.76   30    0.6      4     0.08    1   
                 1213      118       2.36   37   0.74      7     0.14    7   

                      PK GF/GP  PK GA PK GA/GP  SOWGF  SOWGA  
Team             Year                                         
EHC Basel Sharks 0910     0.04     53     1.18      1      2  
                 1011     0.13     38     0.84      1      4  
                 1112      0.2     45        1      0      4  
                 1213     0.16     46     0.92      2      1  
                 1314     0.18     41     0.91      3      2  
EHC Biel         0809     0.06     55      1.1      0      0  
                 0910     0.14     45      0.9      3      3  
                 1011     0.06     47     0.94      4      4  
                 1112     0.02     35      0.7      3      1  
                 1213     0.14     33     0.66      4      3

In [9]:
path_penalties_LNA = "Data/Team/LNA_Regular_Season/penalties/*"
path_penalties_LNB = "Data/Team/LNB_Regular_Season/penalties/*"

df_penalties = []

for file in glob.iglob(path_penalties_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_penalties.append(frame)

for file in glob.iglob(path_penalties_LNB):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_penalties.append(frame)
    
df_penalties = pd.concat(df_penalties)

df_penalties.drop(df_penalties.columns[[0]], axis=1, inplace=True)
df_penalties.reset_index(inplace=True)
df_penalties = df_penalties.drop(df_penalties[df_penalties.Team == 'Durchschnitt'].index)
df_penalties.set_index(['Team','Year'], inplace=True)
df_penalties.sort_index(inplace=True)

df_penalties.head(10)

index  GP  2'   5'  10'  GM  MP  PIM Total
Team             Year                                            
EHC Basel Sharks 0910      7  45  261   0   12   0   0        642
                 1011      8  45  241   3    2   2   2        597
                 1112      9  45  226   1    4   0   1        517
                 1213      8  50  256   2   10   1   1        662
                 1314      2  45  236   3   12   2   1        667
EHC Biel         0809      9  50  321   2    8   4   0        812
                 0910      3  50  303   2   12   1   1        776
                 1011      8  50  226   4    8   4   0        632
                 1112      8  50  216   6    2   5   1        602
                 1213      7  50  239   1    8   1   0        583

In [10]:
path_powerplay_LNA = "Data/Team/LNA_Regular_Season/powerplay/*"
path_powerplay_LNB = "Data/Team/LNB_Regular_Season/powerplay/*"

df_powerplay = []

for file in glob.iglob(path_powerplay_LNA):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerplay.append(frame)

for file in glob.iglob(path_powerplay_LNB):
    frame = pd.read_csv(file,sep=';')
    frame['Year'] = file[39:43]
    df_powerplay.append(frame)

df_powerplay = pd.concat(df_powerplay)

df_powerplay.drop(df_powerplay.columns[[0]], axis=1, inplace=True)
df_powerplay.reset_index(inplace=True)
df_powerplay = df_powerplay.drop(df_powerplay[df_powerplay.Team == 'Durchschnitt'].index)
df_powerplay.set_index(['Team','Year'], inplace=True)
df_powerplay.sort_index(inplace=True)

df_powerplay.head(10)

index  GP  PP OP PP OP/GP  PPG PPG/GP    PP%     PPT  \
Team             Year                                                         
EHC Basel Sharks 0910      9  45    198     4.40   33   0.73  16.67  303:02   
                 1011      8  45    197     4.38   33   0.73  16.75  312:08   
                 1112      8  45    183     4.07   27    0.6  14.75  282:14   
                 1213      6  50    189     3.78   35    0.7  18.52  307:55   
                 1314      1  45    185     4.11   40   0.89  21.62  285:06   
EHC Biel         0809     12  50    261     5.22   29   0.58  11.11  431:52   
                 0910      9  50    259     5.18   44   0.88  16.99  398:40   
                 1011      1  50    201     4.02   39   0.78   19.4  323:04   
                 1112      8  50    198     3.96   30    0.6  15.15  323:53   
                 1213      9  50    231     4.62   37   0.74  16.02  363:29   

                      PP time/GP  PP GA PP GA/GP  
Team             Year                             
EHC Basel Sharks 0910      06:44      8     0.18  
                 1011      06:56      3     0.07  
                 1112      06:16      5     0.11  
                 1213      06:09      4     0.08  
                 1314      06:20      7     0.16  
EHC Biel         0809      08:38     13     0.26  
                 0910      07:58      5      0.1  
                 1011      06:27      6     0.12  
                 1112      06:28      4     0.08  
                 1213      07:16      7     0.14

In [26]:
def timer_to_int(time) :
    m = re.search('(\d+):(\d+)', time)
    if (m) :
        return int(m.group(1))*60 + int(m.group(2))
    else :
        return 0

In [29]:
df = pd.concat([df_shots, df_powerkill, df_goals, df_penalties, df_powerplay], axis=1)
df = df.loc[:,~df.columns.duplicated()]
df = df.reset_index().set_index(['Team','Year'])
df['Number of year'] = 1
df.drop(['GP', 'index', 'SOG', 'S%', 'SOG/GP'],axis=1, inplace=True)
#df.drop(['Neuchâtel'], axis=0, inplace=True)
#df['SOG'] = df['SOG'].apply(lambda x : 0 if x == 0 else int(x[0] + x[2:]))
df['PKT'] = df['PKT'].apply(lambda x : timer_to_int(x))

df['PK time/GP'] = df['PK time/GP'].apply(lambda x : timer_to_int(x))

df['PPT'] = df['PPT'].apply(lambda x : timer_to_int(x))

df['PP time/GP'] = df['PP time/GP'].apply(lambda x : timer_to_int(x))

df.head(5)

#df.to_csv('Data/Team/team_regular_season.csv')

GF  PK SI PK SI/GP  PK GA PK GA/GP    PK%    PKT  \
Team             Year                                                      
EHC Basel Sharks 0910  111    224     4.98     53     1.18  76.34  19961   
                 1011  138    211     4.69     38     0.84  81.99  19849   
                 1112  132    212     4.71     45        1  78.77  18770   
                 1213  138    232     4.64     46     0.92  80.17  21341   
                 1314  144    209     4.64     41     0.91  80.38  19490   

                       PK time/GP  SHG PK GF/GP       ...       10'  GM MP  \
Team             Year                                 ...                    
EHC Basel Sharks 0910         443    2     0.04       ...        12   0  0   
                 1011         441    6     0.13       ...         2   2  2   
                 1112         417    9      0.2       ...         4   0  1   
                 1213         426    8     0.16       ...        10   1  1   
                 1314         433    8     0.18       ...        12   2  1   

                       PIM Total PP OP  PP OP/GP    PP%    PPT PP time/GP  \
Team             Year                                                       
EHC Basel Sharks 0910        642   198      4.40  16.67  18182        404   
                 1011        597   197      4.38  16.75  18728        416   
                 1112        517   183      4.07  14.75  16934        376   
                 1213        662   189      3.78  18.52  18475        369   
                 1314        667   185      4.11  21.62  17106        380   

                       Number of year  
Team             Year                  
EHC Basel Sharks 0910               1  
                 1011               1  
                 1112               1  
                 1213               1  
                 1314               1  

[5 rows x 35 columns]

In [20]:
PO_path_LNA = 'Data/Team/Rankings/By_Match/LNA_Playoff/*'
PO_path_LNB = 'Data/Team/Rankings/By_Match/LNB_Playoff/*'

df_match = []

for file in glob.iglob(PO_path_LNA):
    f = pd.read_csv(file, ';')
    f['Year'] = file[40:44]
    df_match.append(f)

# for file in glob.iglob(PO_path_LNB):
#     f = pd.read_csv(file, ';')
#     f['Year'] = file[40:44]
#     df_match.append(f)


df_match = pd.concat(df_match)
df_match = df_match[['Home', 'Away', 'Resultat', 'Year']]
df_match['Result'] = [ 'h' if x[1] > x[3] else 'a' for x in df_match["Resultat"]]
#df_match['Series_ID'] = df_match['Year']+'_'+set([df_match['Home'], df_match['Away']])
#df_match['Series_ID'] = [0]*len(df_match)

In [27]:
df_match['Series_ID'] = df_match.apply(lambda x: x.Year + '_' + ' '.join(sorted([x.Home, x.Away])), axis=1)
df_match['Opponent 0'] = df_match.apply(lambda x: sorted([x.Home, x.Away])[0], axis=1)
df_match['Opponent 1'] = df_match.apply(lambda x: sorted([x.Home, x.Away])[1], axis=1)
df_match['Result'] = [0 if (x == 'h' and y == a) or (x == 'a' and z == a) else 1 
                 for x, y, z, a in zip(df_match['Result'], df_match['Home'], df_match['Away'], df_match['Opponent 0'])]
df_match[df_match['Year'] == '1516'].head(3)

,Home,Away,Resultat,Year,Result,Series_ID,Opponent 0,Opponent 1
0,HC Lugano,SC Bern,5:4,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern
1,SC Bern,HC Lugano,1:0,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern
2,HC Lugano,SC Bern,2:3,1516,1,1516_HC Lugano SC Bern,HC Lugano,SC Bern


In [22]:
play_off_series = df_match.groupby('Series_ID').agg({
    'Year': {
        'Year': 'first'
    },
    'Home': {
        'Homefield_advantage': lambda x: x.value_counts().index[0],
    },
    'Opponent 0': {
        'Opponent 0': 'first'
    },
    'Opponent 1': {
        'Opponent 1': 'first'
    },
    'Result': {
        'Winner': lambda x: 1 if x.sum() >= 4 else 0
    }
})
play_off_series.columns = play_off_series.columns.droplevel()
play_off_series.head(5)

,Winner,Year,Opponent 1,Homefield_advantage,Opponent 0
Series_ID,,,,,
0809_EV Zug Kloten Flyers,1,0809,Kloten Flyers,Kloten Flyers,EV Zug
0809_EV Zug SC Bern,0,0809,SC Bern,SC Bern,EV Zug
0809_Fribourg-Gottéron HC Davos,1,0809,HC Davos,HC Davos,Fribourg-Gottéron
0809_Fribourg-Gottéron ZSC Lions,0,0809,ZSC Lions,ZSC Lions,Fribourg-Gottéron
0809_Genève-Servette HC Kloten Flyers,1,0809,Kloten Flyers,Kloten Flyers,Genève-Servette HC


In [36]:
df_match_stats = []

for x in play_off_series.iterrows() :
    year = x[1]['Year']
    opp0 = x[1]['Opponent 0']
    opp1 = x[1]['Opponent 1']
    winner = x[1]['Winner']
    
#    line = df.loc[(opp0,year)]-df.loc[(opp1,year)] if winner == opp0 else df.loc[(opp1,year)]-df.loc[(opp0,year)]
    line = df.loc[(opp0,year)]-df.loc[(opp1,year)]
    aa = pd.DataFrame(line).transpose()
    aa['Series_ID'] = year + '_' + ' '.join(sorted([opp0, opp1]))
    aa.set_index(['Series_ID'],inplace=True)

    df_match_stats = pd.concat([pd.DataFrame(df_match_stats),aa])

cc = pd.concat([play_off_series,df_match_stats],axis=1)
cc['Homefield_advantage'] = cc['Homefield_advantage'] == cc['Opponent 1']
cc.Homefield_advantage = cc.Homefield_advantage.astype(int)

cc.to_csv('Data/Ready_for_ML/lnab_playoff.csv')

cc.drop('Homefield_advantage', axis=1).ix[0:5,0:9]

,Winner,Year,Opponent 1,Opponent 0,GF,PK SI,PK SI/GP,PK GA,PK GA/GP
Series_ID,,,,,,,,,
0809_EV Zug Kloten Flyers,1,0809,Kloten Flyers,EV Zug,-3.0,18.0,0.36,4.0,0.08
0809_EV Zug SC Bern,0,0809,SC Bern,EV Zug,-16.0,27.0,0.54,-5.0,-0.10
0809_Fribourg-Gottéron HC Davos,1,0809,HC Davos,Fribourg-Gottéron,-30.0,-25.0,-0.50,11.0,0.22
0809_Fribourg-Gottéron ZSC Lions,0,0809,ZSC Lions,Fribourg-Gottéron,-22.0,31.0,0.62,17.0,0.34
0809_Genève-Servette HC Kloten Flyers,1,0809,Kloten Flyers,Genève-Servette HC,-17.0,26.0,0.52,0.0,0.00
